In [ ]:
import numpy as np
import pandas as pd
import os

np.random.seed(42)

n_days = 365
data = pd.DataFrame({
    "day": range(n_days),
    "daily_demand": np.random.poisson(lam=40, size=n_days)
})

# Create the directory if it doesn't exist
os.makedirs("data", exist_ok=True)
data.to_csv("data/inventory_demand.csv", index=False)

## EOQ (Economic Order Quantity)

In [ ]:
import numpy as np

def eoq(annual_demand, ordering_cost, holding_cost):
    return np.sqrt((2 * annual_demand * ordering_cost) / holding_cost)

## Reorder Point with Safety Stock

In [ ]:
import scipy.stats as stats
import numpy as np

def reorder_point(mean_demand, std_demand, lead_time, service_level=0.95):
    z = stats.norm.ppf(service_level)
    demand_during_lt = mean_demand * lead_time
    std_lt = std_demand * np.sqrt(lead_time)
    return demand_during_lt + z * std_lt

## Inventory Simulation (Core Value)

In [ ]:
def simulate_inventory(demand, Q, ROP, initial_inventory=500):
    inventory = initial_inventory
    stockouts = 0
    orders = 0

    for d in demand:
        inventory -= d
        if inventory < 0:
            stockouts += abs(inventory)
            inventory = 0

        if inventory <= ROP:
            inventory += Q
            orders += 1

    return {
        "total_stockouts": stockouts,
        "orders_placed": orders,
        "ending_inventory": inventory
    }

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/inventory_demand.csv")

D = df["daily_demand"].sum()
mean_d = df["daily_demand"].mean()
std_d = df["daily_demand"].std()

Q = eoq(
    annual_demand=D,
    ordering_cost=300,
    holding_cost=2
)

ROP = reorder_point(
    mean_demand=mean_d,
    std_demand=std_d,
    lead_time=7,
    service_level=0.95
)

results = simulate_inventory(df["daily_demand"], Q, ROP)

print("Optimal EOQ:", round(Q))
print("Reorder Point:", round(ROP))
print(results)

Optimal EOQ: 2073
Reorder Point: 303
{'total_stockouts': 0, 'orders_placed': 7, 'ending_inventory': np.float64(684.8081362636367)}
